In [1]:
import requests
import json
from pprint import pprint
import pandas as pd
import time
import numpy as np
from sklearn.preprocessing import StandardScaler, MinMaxScaler, Normalizer
from sklearn.metrics.pairwise import cosine_similarity

#import spotify keys
from config import client_id
from config import client_secret

# Creating Access Token for Spotify API

In [2]:
def createToken():
    #generate access token for spotify api
    #from: https://stmorse.github.io/journal/spotify-api.html

    AUTH_URL = 'https://accounts.spotify.com/api/token'

    # POST
    auth_response = requests.post(AUTH_URL, {
        'grant_type': 'client_credentials',
        'client_id': client_id,
        'client_secret': client_secret,
    })

    # convert the response to JSON
    auth_response_data = auth_response.json()

    # save the access token
    access_token = auth_response_data['access_token']

    #In order to access, we send a properly formed GET request to the API server, with our access_token in the header. Let’s save this header info now, in the following very specific format:
    headers = {
        'Authorization': 'Bearer {token}'.format(token=access_token)
    }
    
    return headers

# Requesting Input from User

(need to remember to use same variable names when saving user input from project page - search_track, search_artist, region)

In [94]:
# need to add instructions or code to remove punctuations for search to work
search_track = input("Please name a song: ").replace('"', '').replace("'", '').replace("!",'')

Please name a song:  as it was


In [95]:
search_artist = input("Please name that song's artist: ").replace('"', '').replace("'", '')

Please name that song's artist:  harry styles


In [83]:
# options for dropdown: ['Global', 'Vietnam', 'Venezuela', 'Uruguay', 'United Kingdom', 'Ukraine', 'UAE', 'Turkey', 'Thailand', 'Taiwan', 'Switzerland', 'Sweden', 'Spain', 'South Korea', 'South Africa', 'Slovakia', 'Singapore', 'Saudi Arabia', 'Romania', 'Portugal', 'Poland', 'Philippines', 'Peru', 'Paraguay', 'Panama', 'Pakistan', 'Norway', 'Nigeria', 'Nicaragua', 'New Zealand', 'Netherlands', 'Mexico', 'Malaysia', 'Luxembourg', 'Lithuania', 'Latvia', 'Kazakhstan', 'Japan', 'Italy', 'Israel', 'Ireland', 'Indonesia', 'Hungary', 'Hong Kong', 'Honduras', 'Guatemala', 'Greece', 'Germany', 'France', 'Finland', 'Estonia', 'El Salvador', 'Egypt', 'Ecuador', 'Dominican Republic', 'Denmark', 'Czech Republic', 'Cyprus', 'Costa Rica', 'Colombia', 'Chile', 'Canada', 'Bulgaria', 'Brazil', 'Bolivia', 'Belgium', 'Belarus', 'Austria', 'Australia', 'Argentina']
region = input("Please select a region: ")

Please select a region:  South Korea


# Function to Search for Matching Track Using Spotify API, Create DF of Track Features

will be called/used within main function for recommending 5 new songs

In [88]:
def searchTrack(track,artist):
    #base URL for Spotify API search
    search_url = 'https://api.spotify.com/v1/search'
    
    # API search for track & artist input by user
    response = requests.get(search_url + '?q=track%3A' + search_track + '%20artist%3A' + search_artist + '&type=track%2Cartist&limit=1', headers=createToken()).json()
    
    # storing track id, artist, and track name from API search as variables
    track_id = response['tracks']['items'][0]["id"]
    artist_names = response['tracks']['items'][0]["artists"][0]['name']
    track_name = response['tracks']['items'][0]["name"]
    
    #base URL for Spotify API
    base_url = 'https://api.spotify.com/v1/'
    # API call to collect track features
    track_response = requests.get(base_url + 'audio-features/' + track_id, headers=createToken()).json()
    
    #empty lists to store song data retrieved from API call
    danceability = []
    energy = []
    # key = []
    loudness = []
    mode = []
    speechiness = []
    acousticness = []
    instrumentalness = []
    liveness = []
    valence = []
    tempo = []
    id_num = []
    # duration_mins = []
    # time_signature = []

    #fill in track info for each audio feature / empty list
    danceability.append(track_response["danceability"])
    energy.append(track_response["energy"])
    # key.append(track_response["key"])
    loudness.append(track_response["loudness"])
    mode.append(track_response["mode"])
    speechiness.append(track_response["speechiness"])
    acousticness.append(track_response["acousticness"])
    instrumentalness.append(track_response["instrumentalness"])
    liveness.append(track_response["liveness"])
    valence.append(track_response["valence"])
    tempo.append(track_response["tempo"])
    id_num.append(track_response["id"])
    # # duration recorded in api in milliseconds, converting to minutes here
    # duration_mins.append(round((track_response["duration_ms"] / 60000),2))
    # time_signature.append(track_response["time_signature"])    

    #create a dictionary to hold data gathered from the api
    track_dict = {
        "track_id": id_num,
        "artist_names": artist_names,
        "track_name": track_name,
        "danceability": danceability,
        "energy": energy,
        # "key": key,
        "loudness": loudness,
        "mode": mode,
        "speechiness": speechiness,
        "acousticness": acousticness,
        "instrumentalness": instrumentalness,
        "liveness": liveness,
        "valence": valence,
        "tempo": tempo,
        # "duration_mins": duration_mins,
        # "time_signature": time_signature,

    }
    #convert dictionary to a dataframe
    input_track_df = pd.DataFrame(track_dict)
    # input_track_df = input_track_df.drop(columns=['duration_mins'])
    return input_track_df

In [77]:
# test function
input_track_df = searchTrack(search_track,search_artist)
input_track_df

,track_id,artist_names,track_name,danceability,energy,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo
0,4LRPiXqCikLlN15c3yImP7,Harry Styles,As It Was,0.52,0.731,-5.338,0,0.0557,0.342,0.00101,0.311,0.662,173.93


In [89]:
# test function
input_track_df = searchTrack(search_track,search_artist)
input_track_df

,track_id,artist_names,track_name,danceability,energy,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo
0,296nXCOv97WJNRWzIBQnoj,LE SSERAFIM,FEARLESS,0.863,0.62,-7.167,1,0.135,0.0508,0.00324,0.129,0.432,103.971


In [72]:
# test function
input_track_df = searchTrack(search_track,search_artist)
input_track_df

,track_id,artist_names,track_name,danceability,energy,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo
0,4GfK1qOF3uBWidbPlTCQRL,Tulus,Monokrom,0.534,0.462,-9.383,1,0.0326,0.573,0.000006,0.0974,0.519,88.046


In [96]:
# run function to create DF with input track and its audio features
user_track_df = searchTrack(search_track,search_artist)

#import CSVs created in "pulling_chart_track_features.ipynb" and save as individual dfs
features_csv_list = {"Global":"global_df.csv", "Vietnam":"vietnam_df.csv","Venezuela":"venezuela_df.csv",
"Uruguay":"uruguay_df.csv","United Kingdom":"united_kingdom_df.csv","Ukraine":"ukraine_df.csv",
"UAE":"uae_df.csv","Turkey":"turkey_df.csv","Thailand":"thailand_df.csv",
"Taiwan":"taiwan_df.csv","Switzerland":"switzerland_df.csv","Sweden":"sweden_df.csv",
"Spain":"spain_df.csv","South Korea":"south_korea_df.csv","South Africa":"south_africa_df.csv",
"Slovakia":"slovakia_df.csv","Singapore":"singapore_df.csv","Saudi Arabia":"saudi_arabia_df.csv",
"Romania":"romania_df.csv","Portugal":"portugal_df.csv","Poland":"poland_df.csv",
"Philippines":"philippines_df.csv","Peru":"peru_df.csv","Paraguay":"paraguay_df.csv",
"Panama":"panama_df.csv","Pakistan":"pakistan_df.csv","Norway":"norway_df.csv",
"Nigeria":"nigeria_df.csv","Nicaragua":"nicaragua_df.csv","New Zealand":"new_zealand_df.csv",
"Netherlands":"netherlands_df.csv","Mexico":"mexico_df.csv",
"Malaysia":"malaysia_df.csv","Luxembourg":"luxembourg_df.csv","Lithuania":"lithuania_df.csv",
"Latvia":"latvia_df.csv","Kazakhstan":"kazakhstan_df.csv","Japan":"japan_df.csv",
"Italy":"italy_df.csv","Israel":"israel_df.csv","Ireland":"ireland_df.csv",
"Indonesia":"indonesia_df.csv",
"Hungary":"hungary_df.csv","Hong Kong":"hong_kong_df.csv","Honduras":"honduras_df.csv",
"Guatemala":"guatemala_df.csv","Greece":"greece_df.csv","Germany":"germany_df.csv",
"France":"france_df.csv","Finland":"finland_df.csv","Estonia":"estonia_df.csv",
"El Salvador":"el_salvador_df.csv","Egypt":"egypt_df.csv","Ecuador":"ecuador_df.csv",
"Dominican Republic":"dominican_republic_df.csv","Denmark":"denmark_df.csv","Czech Republic":"czech_republic_df.csv",
"Cyprus":"cyprus_df.csv","Costa Rica":"costa_rica_df.csv","Colombia":"colombia_df.csv",
"Chile":"chile_df.csv","Canada":"canada_df.csv","Bulgaria":"bulgaria_df.csv",
"Brazil":"brazil_df.csv","Bolivia":"bolivia_df.csv","Belgium":"belgium_df.csv",
"Belarus":"belarus_df.csv","Austria":"austria_df.csv","Australia":"australia_df.csv",
"Argentina":"argentina_df.csv"}
# currently not working w/ api calls: "morocco":"morocco_df.csv","india":"india_df.csv","iceland":"iceland_df.csv"

d = {}

for key in features_csv_list:
    d[key] = pd.read_csv(f'resources/top_chart_features/{features_csv_list[key]}') 
    d[key] = d[key].drop(columns=['region','key','duration_mins','time_signature'])

selected_chart_df = d['South Korea']

In [97]:
# create combined df of the track features from the input track and all of the tracks from the selected regional chart
# row 0 will be the input song
combined_df = pd.concat([user_track_df,selected_chart_df.loc[:]]).reset_index(drop=True)    
combined_df

,track_id,artist_names,track_name,danceability,energy,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo
0,4LRPiXqCikLlN15c3yImP7,Harry Styles,As It Was,0.520,0.731,-5.338,0,0.0557,0.34200,0.001010,0.3110,0.662,173.930
1,0a4MMyCrzT0En247IhqZbD,NewJeans,Hype Boy,0.593,0.935,-3.430,0,0.2250,0.26800,0.000000,0.2860,0.783,99.976
2,5mdWIwsJAzR97ShGkt8gcR,"ZICO, HOMIES",New thing (Prod. ZICO) (Feat. Homies),0.775,0.865,-2.015,1,0.3070,0.04530,0.000000,0.2610,0.663,175.120
3,7gRFDGEzF9UkBV233yv2dc,BLACKPINK,Shut Down,0.818,0.690,-5.119,1,0.0454,0.00327,0.000000,0.1860,0.688,110.071
4,2gYj9lubBorOPIVWsTXugG,IVE,After LIKE,0.680,0.922,-1.215,1,0.1210,0.10300,0.000000,0.0877,0.799,125.014
...,...,...,...,...,...,...,...,...,...,...,...,...,...
196,03qu1u4hDyepQQi2lNxCka,Lee Mujin,Traffic light,0.609,0.717,-3.547,0,0.0597,0.09650,0.000000,0.0694,0.545,97.026
197,1xWVYPdaLm909DbFmuPGOR,Sandeul,Slightly Tipsy (She is My Type♡ X SANDEUL),0.697,0.447,-6.552,1,0.0264,0.76900,0.000000,0.1150,0.549,93.931
198,0U5HgFCn3EJN2zZqI0KGfJ,Tophyun,Tell me you love me,0.716,0.557,-5.889,1,0.0308,0.77600,0.000000,0.1210,0.625,106.986
199,4fFfyouye6vrX9o9z4PVMu,Lauv,Never Not,0.470,0.221,-9.243,1,0.0371,0.69300,0.000000,0.1040,0.264,133.714


In [98]:
# scale data (only features columns), create df of scaled data
scaler = StandardScaler()
# if using song_duration, needs to be combined_df.iloc[:, 3:16]
chart_scaled = scaler.fit_transform(combined_df.iloc[:, 3:13])
chart_scaled_df = pd.DataFrame(chart_scaled, columns=combined_df.iloc[:, 3:13].columns)
chart_scaled_df

,danceability,energy,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo
0,-0.904772,0.402887,-0.129001,-1.532971,-0.410229,0.203382,-0.082870,1.058591,0.520672,1.597697
1,-0.386432,1.600247,0.820647,-1.532971,1.732061,-0.067792,-0.097737,0.851697,1.114383,-0.755781
2,0.905868,1.189388,1.524919,0.652328,2.769673,-0.883881,-0.097737,0.644802,0.525579,1.635567
3,1.211191,0.162241,-0.020000,0.652328,-0.540563,-1.037901,-0.097737,0.024119,0.648246,-0.434522
4,0.231315,1.523944,1.923095,0.652328,0.416065,-0.672438,-0.097737,-0.789389,1.192890,0.041017
...,...,...,...,...,...,...,...,...,...,...
196,-0.272824,0.320715,0.762414,-1.532971,-0.359614,-0.696258,-0.097737,-0.940836,-0.053412,-0.849660
197,0.352025,-1.264026,-0.733232,0.652328,-0.780985,1.768133,-0.097737,-0.563461,-0.033785,-0.948154
198,0.486935,-0.618391,-0.403244,0.652328,-0.725309,1.793784,-0.097737,-0.513806,0.339124,-0.532698
199,-1.259800,-2.590513,-2.072594,0.652328,-0.645590,1.489629,-0.097737,-0.654494,-1.432196,0.317882


In [103]:
# create array for input track data
array1 = chart_scaled_df.iloc[0,:].to_numpy().reshape(1, -1)
# create array for selected chart
array2 = chart_scaled_df.iloc[1:,:].to_numpy()

# run cosine similarity
cosine_sim = cosine_similarity(array1, array2)

# create list that is ranked by score, descending order
sim_scores = list(enumerate(cosine_sim[-1,:]))
sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

# create empty list to store recommendations
ranked_tracks = []

for i in range(0, 200):
    indx = sim_scores[i][0]
    ranked_tracks.append([combined_df['track_id'].iloc[indx], combined_df['artist_names'].iloc[indx], combined_df['track_name'].iloc[indx], sim_scores[i][1]])
#     np.round(sim_scores[i][1],decimals=5)
ranked_tracks

[['296nXCOv97WJNRWzIBQnoj', 'LE SSERAFIM', 'FEARLESS', 1.0],
 ['2WTHLEVjfefbGoW7F3dXIg', 'aespa', 'Girls', 0.8002512289878987],
 ['2DwUdMJ5uxv20EhAildreg', 'NewJeans', 'Cookie', 0.7198086013564983],
 ['68EkhVWIeULhHxcbi1QhzK',
  'AJR',
  "World's Smallest Violin",
  0.7187699750297063],
 ['3LtpKP5abr2qqjunvjlX5i', 'Central Cee', 'Doja', 0.7077559040090882],
 ['6UelLqGlWMcVH1E5c4H7lY',
  'Harry Styles',
  'Watermelon Sugar',
  0.706978821434817],
 ['3uKoBRThCMtgKss90CiXQc', 'ASH ISLAND', 'Nightmare', 0.6535625582919046],
 ['0M4EDkH7RpbKlWwrZ9o17N', 'An Nyeong', 'haeyo (2022)', 0.6345643411673128],
 ['56s2s5e8WuBsWVKnmz6J9L', 'STAYC', 'BEAUTIFUL MONSTER', 0.6329350953823673],
 ['3gFcGnU4kTdMYLXDjH1TK8', 'STAYC', 'RUN2U', 0.6035582750929768],
 ['6uEWYpv8HNAdbwHlqemG1F', 'BOL4', 'Love story', 0.5370469129496991],
 ['4FKmrrI6y8REWoCyV5tAhY',
  '10cm, BIG Naughty',
  'Just 10 centimeters',
  0.5319014258907724],
 ['0CB3B5ir8I2KbB3dkGVtWF',
  'WSG WANNBE (Gaya-G)',
  'At That Moment',
  0.511

# Function to Recommend 5 Tracks from Selected Top 200 Chart

In [84]:
def recommendSongs(track, artist, region):
    # run function to create DF with input track and its audio features
    user_track_df = searchTrack(track,artist)
    
    #import CSVs created in "pulling_chart_track_features.ipynb" and save as individual dfs
    features_csv_list = {"Global":"global_df.csv", "Vietnam":"vietnam_df.csv","Venezuela":"venezuela_df.csv",
    "Uruguay":"uruguay_df.csv","United Kingdom":"united_kingdom_df.csv","Ukraine":"ukraine_df.csv",
    "UAE":"uae_df.csv","Turkey":"turkey_df.csv","Thailand":"thailand_df.csv",
    "Taiwan":"taiwan_df.csv","Switzerland":"switzerland_df.csv","Sweden":"sweden_df.csv",
    "Spain":"spain_df.csv","South Korea":"south_korea_df.csv","South Africa":"south_africa_df.csv",
    "Slovakia":"slovakia_df.csv","Singapore":"singapore_df.csv","Saudi Arabia":"saudi_arabia_df.csv",
    "Romania":"romania_df.csv","Portugal":"portugal_df.csv","Poland":"poland_df.csv",
    "Philippines":"philippines_df.csv","Peru":"peru_df.csv","Paraguay":"paraguay_df.csv",
    "Panama":"panama_df.csv","Pakistan":"pakistan_df.csv","Norway":"norway_df.csv",
    "Nigeria":"nigeria_df.csv","Nicaragua":"nicaragua_df.csv","New Zealand":"new_zealand_df.csv",
    "Netherlands":"netherlands_df.csv","Mexico":"mexico_df.csv",
    "Malaysia":"malaysia_df.csv","Luxembourg":"luxembourg_df.csv","Lithuania":"lithuania_df.csv",
    "Latvia":"latvia_df.csv","Kazakhstan":"kazakhstan_df.csv","Japan":"japan_df.csv",
    "Italy":"italy_df.csv","Israel":"israel_df.csv","Ireland":"ireland_df.csv",
    "Indonesia":"indonesia_df.csv",
    "Hungary":"hungary_df.csv","Hong Kong":"hong_kong_df.csv","Honduras":"honduras_df.csv",
    "Guatemala":"guatemala_df.csv","Greece":"greece_df.csv","Germany":"germany_df.csv",
    "France":"france_df.csv","Finland":"finland_df.csv","Estonia":"estonia_df.csv",
    "El Salvador":"el_salvador_df.csv","Egypt":"egypt_df.csv","Ecuador":"ecuador_df.csv",
    "Dominican Republic":"dominican_republic_df.csv","Denmark":"denmark_df.csv","Czech Republic":"czech_republic_df.csv",
    "Cyprus":"cyprus_df.csv","Costa Rica":"costa_rica_df.csv","Colombia":"colombia_df.csv",
    "Chile":"chile_df.csv","Canada":"canada_df.csv","Bulgaria":"bulgaria_df.csv",
    "Brazil":"brazil_df.csv","Bolivia":"bolivia_df.csv","Belgium":"belgium_df.csv",
    "Belarus":"belarus_df.csv","Austria":"austria_df.csv","Australia":"australia_df.csv",
    "Argentina":"argentina_df.csv"}
    # currently not working w/ api calls: "morocco":"morocco_df.csv","india":"india_df.csv","iceland":"iceland_df.csv"

    d = {}

    for key in features_csv_list:
        d[key] = pd.read_csv(f'resources/top_chart_features/{features_csv_list[key]}') 
        d[key] = d[key].drop(columns=['region','key','duration_mins','time_signature'])

    selected_chart_df = d[region]
    
    # create combined df of the track features from the input track and all of the tracks from the selected regional chart
    # row 0 will be the input song
    combined_df = pd.concat([user_track_df,selected_chart_df.loc[:]]).reset_index(drop=True)    
    
    # scale data (only features columns), create df of scaled data
    scaler = StandardScaler()
    # if using song_duration, needs to be combined_df.iloc[:, 3:16]
    chart_scaled = scaler.fit_transform(combined_df.iloc[:, 3:13])
    chart_scaled_df = pd.DataFrame(chart_scaled, columns=combined_df.iloc[:, 3:13].columns)

    # create array for input track data
    array1 = chart_scaled_df.iloc[0,:].to_numpy().reshape(1, -1)
    # create array for selected chart
    array2 = chart_scaled_df.iloc[1:,:].to_numpy()

    # run cosine similarity
    cosine_sim = cosine_similarity(array1, array2)
    
    # create list that is ranked by score, descending order
    sim_scores = list(enumerate(cosine_sim[-1,:]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    
    # create empty list to store recommendations
    ranked_tracks = []

    for i in range(0, 5):
        indx = sim_scores[i][0]
        ranked_tracks.append([combined_df['track_id'].iloc[indx], combined_df['artist_names'].iloc[indx], combined_df['track_name'].iloc[indx], np.round(sim_scores[i][1],decimals=2)])
    
    return ranked_tracks

In [85]:
recommendSongs(search_track,search_artist,region)

[['296nXCOv97WJNRWzIBQnoj', 'LE SSERAFIM', 'FEARLESS', 1.0],
 ['2WTHLEVjfefbGoW7F3dXIg', 'aespa', 'Girls', 0.8],
 ['2DwUdMJ5uxv20EhAildreg', 'NewJeans', 'Cookie', 0.72],
 ['68EkhVWIeULhHxcbi1QhzK', 'AJR', "World's Smallest Violin", 0.72],
 ['3LtpKP5abr2qqjunvjlX5i', 'Central Cee', 'Doja', 0.71]]

# (Keep cell below for testing track search)

In [10]:
#base URL for Spotify API search
search_url = 'https://api.spotify.com/v1/search'

response = requests.get(search_url + '?q=track%3A' + search_track + '%20artist%3A' + search_artist + '&type=track%2Cartist&limit=1', headers=createToken()).json()

pprint(response)

# track_id = response['tracks']['items'][0]["id"]
# artist_names = response['tracks']['items'][0]["artists"][0]['name']
# track_name = response['tracks']['items'][0]["name"]

{'artists': {'href': 'https://api.spotify.com/v1/search?query=track%3Asoul+man+artist%3Asam+&type=artist&offset=0&limit=1',
             'items': [],
             'limit': 1,
             'next': None,
             'offset': 0,
             'previous': None,
             'total': 0},
 'tracks': {'href': 'https://api.spotify.com/v1/search?query=track%3Asoul+man+artist%3Asam+&type=track&offset=0&limit=1',
            'items': [{'album': {'album_type': 'album',
                                 'artists': [{'external_urls': {'spotify': 'https://open.spotify.com/artist/2BVYdY4PyfCF9z4NrkhEB2'},
                                              'href': 'https://api.spotify.com/v1/artists/2BVYdY4PyfCF9z4NrkhEB2',
                                              'id': '2BVYdY4PyfCF9z4NrkhEB2',
                                              'name': 'Sam & Dave',
                                              'type': 'artist',
                                              'uri': 'spotify:artist:2BVYdY4Py

# Reading in CSVs of Regional and Global Top 200 Charts

In [11]:
#import CSVs created in "pulling_chart_track_features.ipynb" and save as individual dfs
features_csv_list = {"Global":"global_df.csv", "Vietnam":"vietnam_df.csv","Venezuela":"venezuela_df.csv",
"Uruguay":"uruguay_df.csv","United Kingdom":"united_kingdom_df.csv","Ukraine":"ukraine_df.csv",
"UAE":"uae_df.csv","Turkey":"turkey_df.csv","Thailand":"thailand_df.csv",
"Taiwan":"taiwan_df.csv","Switzerland":"switzerland_df.csv","Sweden":"sweden_df.csv",
"Spain":"spain_df.csv","South Korea":"south_korea_df.csv","South Africa":"south_africa_df.csv",
"Slovakia":"slovakia_df.csv","Singapore":"singapore_df.csv","Saudi Arabia":"saudi_arabia_df.csv",
"Romania":"romania_df.csv","Portugal":"portugal_df.csv","Poland":"poland_df.csv",
"Philippines":"philippines_df.csv","Peru":"peru_df.csv","Paraguay":"paraguay_df.csv",
"Panama":"panama_df.csv","Pakistan":"pakistan_df.csv","Norway":"norway_df.csv",
"Nigeria":"nigeria_df.csv","Nicaragua":"nicaragua_df.csv","New Zealand":"new_zealand_df.csv",
"Netherlands":"netherlands_df.csv","Mexico":"mexico_df.csv",
"Malaysia":"malaysia_df.csv","Luxembourg":"luxembourg_df.csv","Lithuania":"lithuania_df.csv",
"Latvia":"latvia_df.csv","Kazakhstan":"kazakhstan_df.csv","Japan":"japan_df.csv",
"Italy":"italy_df.csv","Israel":"israel_df.csv","Ireland":"ireland_df.csv",
"Indonesia":"indonesia_df.csv",
"Hungary":"hungary_df.csv","Hong Kong":"hong_kong_df.csv","Honduras":"honduras_df.csv",
"Guatemala":"guatemala_df.csv","Greece":"greece_df.csv","Germany":"germany_df.csv",
"France":"france_df.csv","Finland":"finland_df.csv","Estonia":"estonia_df.csv",
"El Salvador":"el_salvador_df.csv","Egypt":"egypt_df.csv","Ecuador":"ecuador_df.csv",
"Dominican Republic":"dominican_republic_df.csv","Denmark":"denmark_df.csv","Czech Republic":"czech_republic_df.csv",
"Cyprus":"cyprus_df.csv","Costa Rica":"costa_rica_df.csv","Colombia":"colombia_df.csv",
"Chile":"chile_df.csv","Canada":"canada_df.csv","Bulgaria":"bulgaria_df.csv",
"Brazil":"brazil_df.csv","Bolivia":"bolivia_df.csv","Belgium":"belgium_df.csv",
"Belarus":"belarus_df.csv","Austria":"austria_df.csv","Australia":"australia_df.csv",
"Argentina":"argentina_df.csv"}
# currently not working w/ api calls: "morocco":"morocco_df.csv","india":"india_df.csv","iceland":"iceland_df.csv"

d = {}

for key in features_csv_list:
    d[key] = pd.read_csv(f'resources/top_chart_features/{features_csv_list[key]}') 
    d[key] = d[key].drop(columns=['region']) # try adding ",'duration_mins'" as well

In [12]:
regions_list = d.keys()
regions_list

dict_keys(['Global', 'Vietnam', 'Venezuela', 'Uruguay', 'United Kingdom', 'Ukraine', 'UAE', 'Turkey', 'Thailand', 'Taiwan', 'Switzerland', 'Sweden', 'Spain', 'South Korea', 'South Africa', 'Slovakia', 'Singapore', 'Saudi Arabia', 'Romania', 'Portugal', 'Poland', 'Philippines', 'Peru', 'Paraguay', 'Panama', 'Pakistan', 'Norway', 'Nigeria', 'Nicaragua', 'New Zealand', 'Netherlands', 'Mexico', 'Malaysia', 'Luxembourg', 'Lithuania', 'Latvia', 'Kazakhstan', 'Japan', 'Italy', 'Israel', 'Ireland', 'Indonesia', 'Hungary', 'Hong Kong', 'Honduras', 'Guatemala', 'Greece', 'Germany', 'France', 'Finland', 'Estonia', 'El Salvador', 'Egypt', 'Ecuador', 'Dominican Republic', 'Denmark', 'Czech Republic', 'Cyprus', 'Costa Rica', 'Colombia', 'Chile', 'Canada', 'Bulgaria', 'Brazil', 'Bolivia', 'Belgium', 'Belarus', 'Austria', 'Australia', 'Argentina'])

In [13]:
len(regions_list)

70

In [14]:
dropdown_list = ['Global', 'Vietnam', 'Venezuela', 'Uruguay', 'United Kingdom', 'Ukraine', 'UAE', 'Turkey', 'Thailand', 'Taiwan', 'Switzerland', 'Sweden', 'Spain', 'South Korea', 'South Africa', 'Slovakia', 'Singapore', 'Saudi Arabia', 'Romania', 'Portugal', 'Poland', 'Philippines', 'Peru', 'Paraguay', 'Panama', 'Pakistan', 'Norway', 'Nigeria', 'Nicaragua', 'New Zealand', 'Netherlands', 'Mexico', 'Malaysia', 'Luxembourg', 'Lithuania', 'Latvia', 'Kazakhstan', 'Japan', 'Italy', 'Israel', 'Ireland', 'Indonesia', 'Hungary', 'Hong Kong', 'Honduras', 'Guatemala', 'Greece', 'Germany', 'France', 'Finland', 'Estonia', 'El Salvador', 'Egypt', 'Ecuador', 'Dominican Republic', 'Denmark', 'Czech Republic', 'Cyprus', 'Costa Rica', 'Colombia', 'Chile', 'Canada', 'Bulgaria', 'Brazil', 'Bolivia', 'Belgium', 'Belarus', 'Austria', 'Australia', 'Argentina']

In [18]:
dropdown_list.sort(reverse=False)

In [19]:
print(dropdown_list)

['Argentina', 'Australia', 'Austria', 'Belarus', 'Belgium', 'Bolivia', 'Brazil', 'Bulgaria', 'Canada', 'Chile', 'Colombia', 'Costa Rica', 'Cyprus', 'Czech Republic', 'Denmark', 'Dominican Republic', 'Ecuador', 'Egypt', 'El Salvador', 'Estonia', 'Finland', 'France', 'Germany', 'Global', 'Greece', 'Guatemala', 'Honduras', 'Hong Kong', 'Hungary', 'Indonesia', 'Ireland', 'Israel', 'Italy', 'Japan', 'Kazakhstan', 'Latvia', 'Lithuania', 'Luxembourg', 'Malaysia', 'Mexico', 'Netherlands', 'New Zealand', 'Nicaragua', 'Nigeria', 'Norway', 'Pakistan', 'Panama', 'Paraguay', 'Peru', 'Philippines', 'Poland', 'Portugal', 'Romania', 'Saudi Arabia', 'Singapore', 'Slovakia', 'South Africa', 'South Korea', 'Spain', 'Sweden', 'Switzerland', 'Taiwan', 'Thailand', 'Turkey', 'UAE', 'Ukraine', 'United Kingdom', 'Uruguay', 'Venezuela', 'Vietnam']
